In [1]:
from __future__ import print_function

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import argparse
import datetime
import shutil
import time
import random
import json

from trainer import Trainer
import utils_funcs
import traceback

In [4]:
epochs=400
rho=5
mixup=1
load = None
load='pretrained_models/single_resnet/last_model_300.pth'

if load is None:
    with open("configs/cp_resnet.json", "r") as text_file:
        default_conf = json.load(text_file)
else:
    with open("configs/cp_resnet_eval.json", "r") as text_file:
        default_conf = json.load(text_file)

default_conf['out_dir'] = default_conf['out_dir'] + str(datetime.datetime.now().strftime('%b%d_%H.%M.%S'))

print("The experiment outputs will be found at: ", default_conf['out_dir'])
tensorboard_write_path = default_conf['out_dir'].replace("out", "runs", 1)
print("The experiment tesnorboard can be accessed: tensorboard --logdir  ", tensorboard_write_path)

print("Rho value : ", rho)
print("Use Mix-up : ", mixup)

from models.cp_resnet import get_model_based_on_rho

default_conf['model_config'] = get_model_based_on_rho(rho, config_only=True)

# find the RF at the 24th layer of the model defined by this config
# this equations are explained in:
# The Receptive Field as a Regularizer in Deep Convolutional Neural Networks for Acoustic Scene Classification,
# Koutini et al.
# EUSIPCO 2019

try:
    # set utils_funcs.model_config to the current model (not safe with lru)
    utils_funcs.model_config = default_conf['model_config']
    _, max_rf = utils_funcs.get_maxrf(24)
    print("For this Rho, the maximium RF is: ", max_rf)
except:
    print("couldn't determine the max RF, maybe non-standard model_config")
    traceback.print_exc()

if mixup:
    default_conf['use_mixup'] = True
    default_conf['loss_criterion'] = 'mixup_default'
else:
    default_conf['use_mixup'] = False

The experiment outputs will be found at:  ./outdir/cp_resnet/exp_Nov06_12.59.49
The experiment tesnorboard can be accessed: tensorboard --logdir   ./runsdir/cp_resnet/exp_Nov06_12.59.49
Rho value :  5
Use Mix-up :  1
For this Rho, the maximium RF is:  81


In [5]:
epochs = epochs
trainer = Trainer(default_conf)
if load is not None:
    model_path = load
    print("will load pre-trained model from ", model_path)
    import torch
    from datetime import datetime
    checkpoint = torch.load(model_path)
    try:
        trainer.bare_model.load_state_dict(checkpoint['state_dict'])
    except:
        print("\n\nFailed: to load weights check that you have the correct rho value\n\n")
        raise

    


0.0
I: for detailed run info use 
 tail -f ./outdir/cp_resnet/exp_Nov06_12.59.49/info.log
I: tensorboard run path: ./runsdir/cp_resnet/exp_Nov06_12.59.49
I: To monitor this experiment use:
 tensorboard --logdir ./runsdir/cp_resnet/exp_Nov06_12.59.49
Network(
  (in_c): Sequential(
    (0): Conv2d(2, 128, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (stage1): Sequential(
    (block1): BasicBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (maxpool1_0): MaxPool2d(kernel_size=2, stride=2, padding=

In [6]:
if load is not None:
    print("model loaded, predicting...")
    sids, propbs = trainer.do_predict("eval",{})
    print(sids[0])
    print(propbs[0])
    print("sids:",len(sids),propbs.shape)
    torch.save((sids, propbs),str(datetime.now())+"eval_predictions.pth")

model loaded, predicting...
type of model: <class 'models.cp_resnet.Network'>
3566612
self.dataloaders  {'testing': <torch.utils.data.dataloader.DataLoader object at 0x7f969482da90>, 'training': <torch.utils.data.dataloader.DataLoader object at 0x7f969482dcc0>, 'eval': <torch.utils.data.dataloader.DataLoader object at 0x7f96951a8080>}
test loader: 7200
x: torch.Size([10, 2, 256, 431])
in_c: torch.Size([10, 128, 127, 215])
stage1: torch.Size([10, 128, 15, 26])
stage2: torch.Size([10, 256, 15, 26])
stage3: torch.Size([10, 512, 15, 26])
feed_forward: torch.Size([10, 10, 1, 1])
logit: torch.Size([10, 10])
tensor([[-0.5738, -1.2893, -2.1230, -0.5314,  6.8445, -0.2276, -1.2672, -0.5017,
         -0.6864, -1.6717],
        [-1.5723,  0.9029,  1.1515, -1.1577, -2.4846,  0.8643, -2.5983, -1.7137,
         -0.8663,  4.1033],
        [-0.5631, -2.8480, -2.3224, -0.4058, -2.8077, -0.5305,  3.4302,  0.1361,
         -1.8396, -3.3523],
        [ 0.2794, -2.7340, -2.7687, -2.9684, -1.8256,  4.6793,  

In [ ]:
# else:
#     trainer.fit(epochs)
#     trainer.predict("last")
#     trainer.load_best_model()
#     trainer.predict()